# step01_Extract heldoutset (txt) from source (excel file) task 7

In [ ]:
import pandas as pd
import os
import glob
import preprocessing

sources = ['../sources/visits_2011to2020/*.xlsx']

DEPARTMENTS = ["내분비대사내과", "호흡기내과", "순환기내과", "소화기내과", "류마티스내과", "신장내과", "알레르기내과", "감염내과"]

docperfile = 50

print("DEPARTMENTS: ", DEPARTMENTS)

In [ ]:
# heldout patients
pools = []
train_pool = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_train.txt"
test_pool  = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_test.txt"

def target_ptnums(target_file):
    target_pool = []
    file = open(target_file, "r")
    lines = file.readlines()
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        ptnum = line.split("/")[-1].replace(".txt", "")
        target_pool.append(ptnum)
    file.close()
    
    return target_pool

pools_train = target_ptnums(train_pool)
pools_test  = target_ptnums(test_pool)
pools = pools_train + pools_test
print("len(pools): ", len(pools))
print(pools[:10])

In [ ]:
# load order of sections (same as in task 5)
# sorted_manually.txt
section_path = "./data/02_type_to_file_links/sections_visits_ordered_2011to2020.txt"
file = open(section_path, "r")
lines = file.readlines()
file.close()

section_to_idx = {}

for l in range(len(lines)):
    line = lines[l].strip("\n")
    dat = line.split("\t")
    if len(dat)<2:
        continue
    doctype_ = dat[0]
    section_ = dat[1]
    target = str(doctype_)+"\t"+str(section_)
    section_to_idx[target] = l
    

### Extract heldoutset as .txt file (task 7)

In [ ]:
for s in range(len(sources)):
    outputdir = sources[s].split("/")[-2]
    output_directory = "./data/"+str(outputdir)
    print("output_directory: ", output_directory)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    filenames = glob.glob(sources[s])
    filenames.sort()
    print(filenames)
    
    document = {}
    for f in range(len(filenames)):
        reading_file = filenames[f].split("/")[-1]
        print("reading_file: ", reading_file)

        print("reading...")
        ws = pd.read_excel(filenames[f], engine='openpyxl')

        print("processing...")
        
        print(str(f) + " / " + str(len(filenames)) + "..." + filenames[f])
        for row in range(len(ws)):
            if (row+1)%1000==0:
                print(str(row) + " / " + str(len(ws)))
            
            outtext = []
            
            pt_id = ws['fake_id'][row]
            date = ws['수진(진료)일'][row]
            dptment = ws["수진(퇴원포함)진료과"][row]
            doctype = ws["서식명"][row]
            
            doctype = preprocessing.text_preprocessing(doctype)        
            doctype = "".join(doctype)
            doctype = doctype.replace("\n", " ")
            
            section = ws['서식항목명'][row]
            section = preprocessing.text_preprocessing(section)
            section = "".join(section)
            section = section.replace("\n", " ")
            
            
            if len(document)>0:    
                if (date != date_last or dptment!=dptment_last or pt_id!=pt_id_last or \
                    doctype!=doctype_last):
                    
                    zeros = "".join(["0"]*(8-len(str(pt_id_last))))
                    pt_id_str = zeros + str(pt_id_last)
                    
                    # patient id was not in heldout set pool
                    if pt_id_str not in pools:
                        """pt_id_current_str is not in heldout set pool"""
                        document = {}
                    
                    # patient id was in heldout set pool
                    else:
                        # output
                        date_last_str = preprocessing.text_preprocessing(date_last)
                        date_last_str = "".join(date_last_str)
                        date_last_str = date_last_str.replace("\n", " ")                    
                        document_list = ["DATE [ " + str(date_last_str) + " ]"] + \
                                        ["DOCTYPE [ " + str(doctype_last) + " ]"]

                        section_to_order = {}
                        for sectionname in document:
                            seckey = str(doctype_last)+"\t"+str(sectionname)
                            vl = section_to_idx[seckey]
                            section_to_order[sectionname] = vl

                        # sort section
                        section_to_order = dict(sorted(section_to_order.items(), key=lambda x: x[1]))
                        for sectionname in section_to_order:
                            document_list.append("SECTION [ "+sectionname+" ]")
                            document_list.append(document[sectionname])

                        target_folder = output_directory+"/"+str(dptment_last)+"/"+str(((pt_id_last//docperfile)+1)*docperfile)
                        if not os.path.exists(target_folder):
                            os.makedirs(target_folder)
                            
                        file = open(target_folder+"/"+pt_id_str+".txt", "a")
                        file.write("\n".join(document_list)+"\n\n")
                        file.close()

                        document = {}

            
            content = ws['서식내용'][row]
            content = preprocessing.text_preprocessing(content)
    
            # Collect Sections & contents
            # heldout set patients only
            zeros_cur = "".join(["0"]*(8-len(str(pt_id))))
            pt_id_current_str = zeros_cur + str(pt_id)
            if pt_id_current_str not in pools:
                """pt_id_current_str is not in heldout set pool"""
            else:
                #document.append("SECTION [ " + str(section) + " ]")
                for c in range(len(content)):
                    content_sub = content[c].split("\n")
                    for s in range(len(content_sub)):
                        if content_sub[s]!="":
                            keysec = str(section).strip()
                            if keysec not in document:
                                document[keysec] = content_sub[s]
                            else:
                                document[keysec] = document[keysec]+"\n"+content_sub[s]
                            
            
            date_last = date
            dptment_last = dptment
            pt_id_last= pt_id
            doctype_last = doctype
            
    
        if len(document)!=0:
            # patient id was not in heldout set pool
            if pt_id_str not in pools:
                """pt_id_current_str is not in heldout set pool"""
                document = {}

            # patient id was in heldout set pool
            else:
                date_last_str = preprocessing.text_preprocessing(date_last)
                date_last_str = "".join(date_last_str)
                date_last_str = date_last_str.replace("\n", " ")                    
                document_list = ["DATE [ " + str(date_last_str) + " ]"] + \
                                ["DOCTYPE [ " + str(doctype_last) + " ]"]

                # sort sections
                section_to_order = {}
                for sectionname in document:
                    seckey = str(doctype_last)+"\t"+str(sectionname)
                    vl = section_to_idx[seckey]
                    section_to_order[sectionname] = vl

                section_to_order = dict(sorted(section_to_order.items(), key=lambda x: x[1]))

                for sectionname in section_to_order:
                    document_list.append("SECTION [ "+sectionname+" ]")
                    document_list.append(document[sectionname])

                target_folder = output_directory+"/"+str(dptment_last)+"/"+str(((pt_id_last//docperfile)+1)*docperfile)
                if not os.path.exists(target_folder):
                    os.makedirs(target_folder)

                zeros = "".join(["0"]*(8-len(str(pt_id_last))))
                pt_id_str = zeros + str(pt_id_last)

                file = open(target_folder+"/"+pt_id_str+".txt", "a")
                file.write("\n".join(document_list)+"\n\n")
                file.close()

                document = {}

#         break

print("Done")